In [5]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.impute import SimpleImputer
train=pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/train.csv')
test= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/test.csv')
sample_submission= pd.read_csv('/kaggle/input/ml-olympiad-co2-emissions-prediction-challenge/sample_submission.csv')

In [6]:
#All country name into unique_values
unique_values = test['Country Name'].unique()
unique_values.shape

(266,)

In [7]:
train=train.drop(columns=['Country Code','Country Name'],axis =1 )
test=test.drop(columns=['Country Name'],axis =1 )

# train data 

In [9]:
import numpy as np

# Calculate the number of iterations needed
num_iterations = len(train) // 12

# List to store transposed DataFrames
transposed_dfs = []

# Iterate through the DataFrame in chunks of 12 rows
for i in range(num_iterations):
    start_idx = i * 12
    end_idx = (i + 1) * 12
    subset_df = train.iloc[start_idx:end_idx]
    transposed_subset_df = subset_df.transpose()
    transposed_dfs.append(transposed_subset_df)
    transposed_subset_df.columns = range(12)

# Concatenate the transposed DataFrames
final_df = pd.concat(transposed_dfs)

# Reset the index
final_df.reset_index(drop=True, inplace=True)




# Filter out rows containing the specified string
final_df = final_df[~final_df.apply(lambda row: 'Access to electricity (% of population)' in row.values, axis=1)]

# Reset the index after dropping rows
final_df.reset_index(drop=True, inplace=True)


column_names = [
    'Access to electricity (% of population)',
    'Agricultural land (% of land area)',
    'Annual freshwater withdrawals, total (% of internal resources)',
    'Arable land (% of land area)',
    'Forest area (% of land area)',
    'Electric power consumption (kWh per capita)',
    'Energy use (kg of oil equivalent per capita)',
    'Renewable electricity output (% of total electricity output)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Population growth (annual %)',
    'GDP per capita (current US$)',
    'CO2 emissions (metric tons per capita)'
]

# Rename columns of final_df
final_df.columns = column_names


final_df = final_df.apply(pd.to_numeric, errors='coerce')

final_df = final_df.astype(float)
final_df['Country Name'] = ''




for i in range(len(final_df) // 16):
    start_idx = i * 16
    end_idx = (i + 1) * 16
    final_df.loc[start_idx:end_idx - 1, 'Country Name'] = unique_values[i]

# drop columns 
final_df= final_df.drop(columns=[
#     'Access to electricity (% of population)',
    'Agricultural land (% of land area)', 
    'Annual freshwater withdrawals, total (% of internal resources)',                         
    'Arable land (% of land area)', 'Forest area (% of land area)',                       
    'Electric power consumption (kWh per capita)', 
    'Energy use (kg of oil equivalent per capita)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Renewable electricity output (% of total electricity output)',                   
    'Population growth (annual %)',
    'GDP per capita (current US$)'                 
                                  
                                  
                                 ],axis =1 )

# imputed use korlam 
final_dff = final_df.drop('Country Name' , axis =1 )
imputer = SimpleImputer(strategy='mean')

final_dff = pd.DataFrame(imputer.fit_transform(final_dff), columns=final_dff.columns)

final_dff['Country Name']=final_df['Country Name']

final_df=final_dff

In [10]:
final_df.isnull().sum()

Access to electricity (% of population)    0
CO2 emissions (metric tons per capita)     0
Country Name                               0
dtype: int64

# Test data convert 

In [11]:
import numpy as np

# Calculate the number of iterations needed
num_iterations = len(test) // 11

# List to store transposed DataFrames
transposed_dfs = []

# Iterate through the DataFrame in chunks of 11 rows
for i in range(num_iterations):
    start_idx = i * 11
    end_idx = (i + 1) * 11
    subset_df = test.iloc[start_idx:end_idx]
    transposed_subset_df = subset_df.transpose()
    transposed_dfs.append(transposed_subset_df)
    transposed_subset_df.columns = range(11)

# Concatenate the transposed DataFrames
final_test_df = pd.concat(transposed_dfs)

# Reset the index
final_test_df.reset_index(drop=True, inplace=True)






# Filter out rows containing the specified string
final_test_df = final_test_df[~final_test_df.apply(lambda row: 'Access to electricity (% of population)' in row.values, axis=1)]

# Reset the index after dropping rows
final_test_df.reset_index(drop=True, inplace=True)






column_names = [
    'Access to electricity (% of population)',
    'Agricultural land (% of land area)',
    'Annual freshwater withdrawals, total (% of internal resources)',
    'Arable land (% of land area)',
    'Forest area (% of land area)',
    'Electric power consumption (kWh per capita)',
    'Energy use (kg of oil equivalent per capita)',
    'Renewable electricity output (% of total electricity output)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Population growth (annual %)',
    'GDP per capita (current US$)',
]

# Rename columns of final_df
final_test_df.columns = column_names


final_test_df = final_test_df.apply(pd.to_numeric, errors='coerce')

final_test_df = final_test_df.astype(float)
final_test_df['Country Name'] = ''






for i in range(len(final_test_df) // 5):
    start_idx = i * 5
    end_idx = (i + 1) * 5
    final_test_df.loc[start_idx:end_idx - 1, 'Country Name'] = unique_values[i]

# drop columns 
final_test_df = final_test_df.drop(columns=[
#     'Access to electricity (% of population)',
    'Agricultural land (% of land area)', 
    'Annual freshwater withdrawals, total (% of internal resources)',                         
    'Arable land (% of land area)', 'Forest area (% of land area)',                       
    'Electric power consumption (kWh per capita)', 
    'Energy use (kg of oil equivalent per capita)',
    'Renewable energy consumption (% of total final energy consumption)',
    'Renewable electricity output (% of total electricity output)',                   
    'Population growth (annual %)',
    'GDP per capita (current US$)'                 
],axis =1 )



# imputed use korlam 
final_dff = final_test_df.drop('Country Name' , axis =1 )
imputer = SimpleImputer(strategy='mean')
final_dff = pd.DataFrame(imputer.fit_transform(final_dff), columns=final_dff.columns)

final_dff['Country Name']=final_test_df['Country Name']

final_test_df=final_dff


# country find 

# for >1 col 

In [14]:
from statsmodels.tsa.arima.model import ARIMA

# Define a dictionary to store the model fit for each country
model_fits = {}

# Iterate over unique countries
for country in unique_values:
    # Filter data for the current country
    afg = final_df[final_df['Country Name'] == country].drop('Country Name', axis=1)
    
    # Extract features and target variable
    country_data = afg.drop(columns=['CO2 emissions (metric tons per capita)']).astype(float)
    co2_emissions = afg['CO2 emissions (metric tons per capita)'].astype(float)
    
    test_data_country = final_test_df[final_test_df['Country Name'] == country].drop('Country Name', axis=1)
    
    try:
        # Replace infinite values with NaN
        test_data_country.replace([np.inf, -np.inf], np.nan, inplace=True)
        # Fill NaN values with the mean of each column
        test_data_country.fillna(test_data_country.mean(), inplace=True)
        
        country_data.replace([np.inf, -np.inf], np.nan, inplace=True)
        country_data.fillna(country_data.mean(), inplace=True)  # Replace NaNs with mean
        
        
        
        # Check if the model has already been fitted for this country
        if country in model_fits:
            # Use the previous model fit's order to fine-tune the model
            model_fit = model_fits[country]
            order = model_fit.model_orders
        else:
            # Otherwise, start with an initial order of (1, 1, 0)
            order = (1, 1, 0)
        
        # Train the ARIMA model with both columns as exogenous variables
        model = ARIMA(co2_emissions, exog=country_data, order=order)
        model_fit = model.fit()
        
        # Update the model fit for the current country
        model_fits[country] = model_fit
        
        # Forecast CO2 emissions using the test data as exogenous variables
        forecast = model_fits[country].forecast(steps=5, exog=test_data_country)
        forecast1 = model_fit.forecast(steps=16, exog=country_data.iloc[-16:])

        # Check if all forecast values are zero
        if all(value == 0 for value in forecast) and all(value == 0 for value in forecast1):
            continue  # Skip this country if all forecast values are zero
        
        # Find the row index in the sample_submission DataFrame
        row_index = sample_submission[sample_submission.eq(country).any(axis=1)].index[0]
        
        # Update the sample_submission DataFrame with the forecast values
        new_values = [country, forecast[16], forecast[17], forecast[18], forecast[19], forecast[20],forecast1[30]]
        sample_submission.loc[row_index] = new_values
    except Exception as e:
        print(f"Error occurred for {country}: {e}")


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this

In [15]:
import pandas as pd

# Initialize an empty list to store concatenated DataFrames for each country
concatenated_dfs = []

for country in unique_values:
    # Filter data for the current country
    predict = sample_submission[sample_submission['Country Name'] == country].drop('Country Name', axis=1)
    predict = pd.DataFrame(predict)
    predict = predict.transpose()
    predict.reset_index(drop=True, inplace=True)
    predict.columns = ['Value']
    predict = predict.drop(predict.index[-1])

    predict1 = final_df[final_df['Country Name'] == country].drop(columns=['Country Name','Access to electricity (% of population)'], axis=1) 
    merged_col = pd.concat([predict1['CO2 emissions (metric tons per capita)'], predict['Value']], axis=0)
    
    # Append the concatenated DataFrame for the current country to the list
    concatenated_dfs.append(merged_col)

# Concatenate all DataFrames in the list into a single DataFrame
concatenated_df = pd.concat(concatenated_dfs, ignore_index=True)
concatenated_df = concatenated_df.to_frame()
concatenated_df = concatenated_df.rename(columns={0: 'co2'})




In [16]:
concatenated_df

,co2
0,0.055167
1,0.055293
2,0.066810
3,0.073005
4,0.054867
...,...
5581,4.662416
5582,4.717067
5583,4.777925
5584,4.799011


In [17]:
concatenated_df['Country Name'] = ''




for i in range(len(concatenated_df) // 21):
    start_idx = i * 21
    end_idx = (i + 1) * 21
    concatenated_df.loc[start_idx:end_idx - 1, 'Country Name'] = unique_values[i]

In [18]:
concatenated_df

,co2,Country Name
0,0.055167,Afghanistan
1,0.055293,Afghanistan
2,0.066810,Afghanistan
3,0.073005,Afghanistan
4,0.054867,Afghanistan
...,...,...
5581,4.662416,World
5582,4.717067,World
5583,4.777925,World
5584,4.799011,World


In [19]:
for country in unique_values:
    # Filter data for the current country
    afg = concatenated_df[concatenated_df['Country Name'] == country].drop('Country Name', axis=1)
    country_data = afg['co2'].astype(float)
    
    try:
        # Fit the ARIMA model
        model = ARIMA(country_data, order=(1, 1, 0))  # Adjust order as needed
        model_fit = model.fit()

        # Forecast next 10 values
        forecast = model_fit.forecast(steps=10)

        # Find the row index in the sample_submission DataFrame
        row_index = sample_submission[sample_submission.eq(country).any(axis=1)].index[0]
        
        # Replace the last value in the sample_submission DataFrame with the forecasted value
        sample_submission.iloc[row_index, -1] = forecast[30]
    except:
        print(f"Error occurred for {country}: Unable to fit ARIMA model")


/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integ

In [20]:
forecast 

21    4.825618
22    4.827591
23    4.828207
24    4.828399
25    4.828458
26    4.828477
27    4.828483
28    4.828485
29    4.828485
30    4.828486
Name: predicted_mean, dtype: float64

In [21]:
sample_submission

,Country Name,2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2030 [YR2030]
0,Afghanistan,0.254286,0.251780,0.256860,0.250110,0.249870,0.249759
1,Albania,1.663645,1.663035,1.654109,1.654106,1.654106,1.654106
2,Algeria,4.028284,4.066659,4.083600,4.110450,4.099710,4.088919
3,American Samoa,4.255368,4.255368,4.255368,4.255368,4.255368,4.255368
4,Andorra,6.491266,6.492024,6.492129,6.492144,6.492146,6.492146
...,...,...,...,...,...,...,...
261,Sub-Saharan Africa,0.789942,0.777281,0.786493,0.783085,0.786414,0.785198
262,Sub-Saharan Africa (excluding high income),0.789569,0.776886,0.786126,0.782711,0.786054,0.784833
263,Sub-Saharan Africa (IDA & IBRD countries),0.789942,0.777281,0.786493,0.783085,0.786414,0.785198
264,Upper middle income,5.645895,5.574935,5.517432,5.473499,5.437150,5.306825


In [22]:
sample_submission.to_csv('forcast[30]_imputed.csv', index=False)